In [ ]:
from threading import Lock
import warnings
import matplotlib.pyplot as plt
import pyximport
from tqdm import tnrange, tqdm_notebook

warnings.filterwarnings('ignore')

import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), os.pardir))
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

pyximport.install(setup_args={"include_dirs": np.get_include()})
import XXZZham as XXZZham
from XXZZham import add_high_energies, rotate_to_00_base
import random
import adiabatic_sim as asim
import time

import multiprocessing
import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(multiprocessing.cpu_count())))
import os
import LH_tools as LHT

%precision %.15f
PRECISION  = 2 ** -40

from IPython.display import display
import FD_tools as FDT

In [3]:
def _simulate_time_p_and_pfab_repeat(repeats: int, T: float, steps: int, psi0: Qobj, psi1: Qobj, H0: Qobj, H1: Qobj):

    """
        Callback for simulation of fowrard and backword propagation
    :param T:
    :param steps:
    :param psi0:
    :param psi1:
    :param H0:
    :param H1:
    :return:
    """
    tlist = np.linspace(0, T, steps)
    backprop_t, prop_t = FDT.create_back_and_forward_props(tlist, H0, H1)
    psi_forward = prop_t * psi0
    
    psi_repeat = (backprop_t*prop_t)**repeats * psi0
    
    Pf0 = abs(psi1.overlap(psi_forward)) ** 2
    Prepeat = abs(psi0.overlap(psi_repeat)) ** 2
    return T, Pf0, Prepeat

In [ ]:
def find_min_time_for_epsilon_and_repeats(final_times:arr,
                                          psi0n:Qobj,
                                          psi1n:Qobj,
                                          H0n:Qobj,
                                          H1n:Qobj,
                                          repeats: int,
                                          epsilon=1 / 10) -> (float, float):
    old_Prepeat = 1 
    old_T = 0
    for T in final_times: 
            _,_, Prepeat = simulate_time_p_and_pfab_repeat(repeats, T, max(T*5,50) , psi0n, psi1n , H0n, H1n)
            if Prepeat > old_Prepeat : 
                print("BAD: At repeats:{} got Prepeat = {} T = {} total time = {}, epsilon = {} not met in first decrease".
                      format(repeats,old_Prepeat,old_T,old_T*2*repeats,epsilon))
                return 
            if Prepeat <= epsilon : 
                print("GOOD: At repeats:{} got Prepeat = {} T = {} total time = {}".
                      format(repeats,old_Prepeat,T,T*2*repeats,epsilon))
                return
            old_Prepeat = Prepeat
            old_T = T 

In [ ]:
type(np.linspace(1,10,10))